In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### 1. Задача – найти все произведения, связанные с подсолнухами. Выполним поиск по слову "sunflowers".

In [2]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()

Сохраним id полученных объектов в список sunflower_ids

In [3]:
sunflower_ids = res['objectIDs']

#### 2. Соберем информацию об этих объектах

Выполним запрос для объекта 437980 для проверки

In [7]:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_test = r_test.json()

In [8]:
obj_test

{'objectID': 437980,
 'isHighlight': False,
 'accessionNumber': '49.30',
 'accessionYear': '1949',
 'isPublicDomain': True,
 'primaryImage': 'https://images.metmuseum.org/CRDImages/ep/original/DP130999.jpg',
 'primaryImageSmall': 'https://images.metmuseum.org/CRDImages/ep/web-large/DP130999.jpg',
 'additionalImages': [],
 'constituents': [{'constituentID': 161947,
   'role': 'Artist',
   'name': 'Vincent van Gogh',
   'constituentULAN_URL': 'http://vocab.getty.edu/page/ulan/500115588',
   'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q5582',
   'gender': ''}],
 'department': 'European Paintings',
 'objectName': 'Painting',
 'title': 'Cypresses',
 'culture': '',
 'period': '',
 'dynasty': '',
 'reign': '',
 'portfolio': '',
 'artistRole': 'Artist',
 'artistPrefix': '',
 'artistDisplayName': 'Vincent van Gogh',
 'artistDisplayBio': 'Dutch, Zundert 1853–1890 Auvers-sur-Oise',
 'artistSuffix': '',
 'artistAlphaSort': 'Gogh, Vincent van',
 'artistNationality': 'Dutch',
 'artistB

#### 3. Напишем цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге. Результаты запишем в датафрейм.

In [9]:
culture_df = pd.DataFrame({'objectID': [], 'title': [],
                           'artistDisplayName': [], 'department': [],
                           'objectBeginDate': [], 'objectEndDate': [],
                           'period': [], 'objectName': [], 'culture': []})

In [10]:
%%time
for i in sunflower_ids:
    r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{i}')
    obj_test = r_test.json()
    try:
        df = pd.DataFrame({c: [obj_test[c]] for c in culture_df.columns})
        culture_df = pd.concat([culture_df, df])
    except:
        print(f'Object {i} is empty.')

Object 696949 is empty.
Object 704667 is empty.
Object 839296 is empty.
Object 841928 is empty.
Object 631808 is empty.
CPU times: user 3.22 s, sys: 94 ms, total: 3.31 s
Wall time: 24.2 s


In [11]:
culture_df.head()

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,436524.0,Sunflowers,Vincent van Gogh,European Paintings,1887.0,1887.0,,Painting,
0,484935.0,Sunflowers,Edward McKnight Kauffer,Modern and Contemporary Art,1921.0,1921.0,,Painting,
0,437112.0,Bouquet of Sunflowers,Claude Monet,European Paintings,1881.0,1881.0,,Painting,
0,210191.0,Vase with sunflowers,Rozenburg Plateelfabriek,European Sculpture and Decorative Arts,1896.0,1896.0,,Vase,
0,431264.0,Sunflowers,Lily Converse,Drawings and Prints,1941.0,1941.0,,Print,


#### 4. Выполним поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишем id объектов в cat_ids. Используя цикл из шага 3, чтобы получить данные о первой 1000 объектов из списка.

In [12]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()

In [13]:
cat_ids = cats['objectIDs']

In [14]:
cats_df = pd.DataFrame({'objectID': [], 'title': [],
                           'artistDisplayName': [], 'department': [],
                           'objectBeginDate': [], 'objectEndDate': [],
                           'period': [], 'objectName': [], 'culture': []})

In [15]:
%%time
for i in cat_ids[1000:]:
    c_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{i}')
    objc_test = c_test.json()
    try:
        df3 = pd.DataFrame({c: [objc_test[c]] for c in cats_df.columns})
        cats_df = pd.concat([cats_df, df3])
    except:
        print(f'Object {i} is empty.')

CPU times: user 1min 15s, sys: 2.77 s, total: 1min 18s
Wall time: 7min 35s


In [16]:
cats_df.head()

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,47150.0,Covered censer In form of Shishi,,Asian Art,1780.0,1780.0,Edo period (1615–1868),Covered censer,Japan
0,47261.0,Figure of a Cow,,Asian Art,1830.0,1830.0,Edo period (1615–1868),Figure,Japan
0,47141.0,Covered bowl,,Asian Art,1750.0,1750.0,Edo period (1615–1868),Covered bowl,Japan
0,47180.0,Plate,,Asian Art,1800.0,1800.0,Edo period (1615–1868),Plate,Japan
0,47304.0,Bowl,,Asian Art,1780.0,1780.0,Edo period (1615–1868),Bowl,Japan


In [17]:
cats_df.groupby('period', as_index=False)\
    .agg({'objectID':'sum'})\
    .sort_values('objectID', ascending=False)

,period,objectID
17,Edo period (1615–1868),95935991.0
0,,55498593.0
56,Ming dynasty (1368–1644),10370877.0
78,Muromachi period (1392–1573),7275851.0
28,Heisei period (1989–2019),6767324.0
...,...,...
52,Ming (1368–1644) and Qing (1644–1911) dynasties,37396.0
76,Muromachi (1392–1573) or Momoyama period (1573...,36110.0
107,Song dynasty (960–1279,36064.0
110,Southern Song (1127–1279) or Yuan (1271–1368) ...,36033.0


В Edo period (1615–1868) было выполнено наибольшее количество работ